
## Calibration tests
### Pose2Sim calibration.py functions
### KaggleHub dataset "danielwe14/stereocamera-chessboard-pictures"

Auteurs : Léa Drolet-Roy

Création : 2025-02-14

Dernière modification :

In [37]:
from pose2sim_trampo import calibration
import os

In [38]:
kaggle_dataset = 'kaggle/data'
imgs = 'kaggle/data/imgs'
rcam = r'kaggle\data\imgs\rightcamera'
lcam = r'kaggle\data\imgs\leftcamera'

In [39]:
intrinsics_config_dict = {'calculate_intrinsics':True, 'overwrite_intrinsics':True, 'show_detection_intrinsics':False, 'intrinsics_method':'board',
                          'intrinsics_extension': 'png', 'extract_every_N_sec':1, 'intrinsics_corners_nb':[11,7], 'intrinsics_square_size':30,
                          'board': {'intrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'intrinsics_corners_nb': [11, 7], 'intrinsics_square_size': 30}}

extrinsics_config_dict = {'calculate_extrinsics':True, 'show_detection_extrinsics':False, 'extrinsics_method':'board',
                          'extrinsics_extension': 'png', 'extrinsics_corners_nb':[11,7], 'extrinsics_square_size':30,
                          'board': {'extrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'extrinsics_corners_nb': [11, 7], 'extrinsics_square_size': 30}}

calib_dir = rcam
calibration.calib_calc_fun(calib_dir, intrinsics_config_dict, extrinsics_config_dict)

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'solvePnP'
> Overload resolution failed:
>  - imagePoints is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'imagePoints'


In [36]:
import numpy as np
imgp = np.zeros((77,1,2))
print(imgp == [])

ValueError: operands could not be broadcast together with shapes (77,1,2) (0,) 